A ANÁLISE DE SENTIMENTOS E EMOÇÕES DE MENSAGENS DE REDES SOCIAIS ENVOLVENDO SERVIÇOS DE ENTREGA DE ALIMENTOS



*   Autor/pesquisador: João Pedro Rodrigues Alves;
*   Instituição: Universidade Presbiteriana Mackenzie;
*   Curso: Ciência da Computação (FCI);
*   Orientador: Mario Olimpio de Menezes.

Código fonte do modelo de aprendizagem de máquina para análise de sentimentos e emoções de clientes dos apps iFood, Rappi e Zé Delivery. O modelo foi alcançado a partir de um estudo aprofundado em um projeto de TCC de mesmo título. Para utilização faça uploud da base de dados no diretório do notebook ou faça as alterações necessárias para alterar o modo de acessar os dados (atente-se ao nome do arquivo e aos nomes das colunas para evitar conflito de nomes com o código fornecido abaixo).

In [ ]:
import re
import nltk
import string
import unicodedata
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import RSLPStemmer

In [ ]:
# Pega atual diretório de trabalho deste notebook e armazena-o em wd
wd = os.getcwd()
print(wd)

# Lista os arquivos na pasta dados
if not os.path.exists(wd):
  os.makedirs(wd)
os.listdir(wd)

In [ ]:
# Abre o Workbook e mostra o nome das planilhas
file = 'Base de Dados_TCC.xlsx'
df_base = pd.ExcelFile(file)
print(df_base.sheet_names)

df_dados = df_base.parse('Sheet1')
df_dados.info()

In [ ]:
# Converte todo o texto das classificações para minúsculo
df_dados['Feeling'] = df_dados['Feeling'].str.lower()
df_dados['Emoção'] = df_dados['Emoção'].str.lower()
df_dados['App'] = df_dados['App'].str.lower()
df_dados.head()

In [ ]:
# Função de geral de limpeza
def limpar_texto(texto):
    # Remove as citações de perfis (começado com @)
    texto = re.sub(r'@[\w\d_]+', ' ', texto)

    # Remove as hastags (começado com #)
    texto = re.sub(r'#[\w\d_]+', '', texto)

    # Remove a string RT isolada
    texto = re.sub(r'\bRT\b', '', texto)

    # Remove links de sites
    texto = re.sub(r'http\S+', '', texto)

    # Converte todo o texto para minúsculo
    texto = texto.lower()

    # Remove pontuações, emojis e caracteres especiais
    texto = re.sub(r'[^\w\s]|_', '', texto, flags=re.UNICODE)

    # Remove acentos
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')

    # Remove números
    texto = re.sub(r'\d+', '', texto)

    # Remove espaços extras
    texto = re.sub(r'\s+', ' ', texto).strip()

    return texto

In [ ]:
# Aplica a função de limpeza à coluna 'Tweet'
df_dados['Clean Tweet'] = df_dados['Tweet'].apply(limpar_texto)
df_dados.tail()

In [ ]:
# Função de tokenização e stemização
def normalize_text(text):
    tokens = word_tokenize(text.lower(), language='portuguese')
    stemmer = RSLPStemmer()
    normalized_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(normalized_tokens)

In [ ]:
# Download de pacotes necesários para aplicar a função
nltk.download('punkt')
nltk.download('rslp')
nltk.download('punkt_tab')

In [ ]:
# Aplica a função de tokenização e stemização
df_dados['Tweet Normalization'] = df_dados['Clean Tweet'].apply(normalize_text)
df_dados.head()

Analíse de Sentimentos

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter

# Separando entrada e saída do modelo
X = df_dados['Tweet Normalization']
y = df_dados['Feeling']

# Criando a matriz de features usando TF-IDF
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X)

# Imprimindo a distribuição original das classes
print('Distribuição original das classes:', Counter(y))

# Divide os dados em conjunto de treino e teste
X_train_smote, X_test, y_train_smote, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)
print('Distribuição original das classes treino:', Counter(y_train_smote))
print('Distribuição original das classes teste:', Counter(y_test))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Cria o modelo de Suport Vector Machine (SVM) com kernel linear
modelo_sentimento = SVC(kernel='linear')

# Treina o modelo
modelo_sentimento.fit(X_train_smote, y_train_smote)

# Realiza a previsão
y_pred = modelo_sentimento.predict(X_test)

# Imprime as métricas de desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
print("Accuracy linear sentimento:    ", accuracy)
print("precision linear sentimento:    ", precision)
print("recall linear sentimento:    ", recall)

# Gera a matriz de confusão
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)

# Imprime o relatório de classificação
print(classification_report(y_test, y_pred))

Analíse de Emoções

In [ ]:
# Separando entrada e saída do modelo
X = df_dados['Tweet Normalization']
y = df_dados['Emoção']

# Criando a matriz de features usando TF-IDF
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X)

# Imprimindo a distribuição original das classes
print('Distribuição original das classes:', Counter(y))

# Divide os dados em conjunto de treino e teste
X_train_smote, X_test, y_train_smote, y_test = train_test_split(X_tfidf, y, test_size=0.4, random_state=42)
print('Distribuição original das classes treino:', Counter(y_train_smote))
print('Distribuição original das classes teste:', Counter(y_test))

In [ ]:
# Cria o modelo de Suport Vector Machine (SVM) com kernel linear
modelo_emocao = SVC(kernel='linear')

# Treina o modelo
modelo_emocao.fit(X_train_smote, y_train_smote)

# Realiza a previsão
y_pred = modelo_emocao.predict(X_test)

# Imprime as métricas de desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
print("Accuracy linear emoção:    ", accuracy)
print("precision linear emoção:    ", precision)
print("recall linear emoção:    ", recall)

# Gera a matriz de confusão
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)

# Imprime o relatório de classificação
print(classification_report(y_test, y_pred))